In [1]:
import cv2
import colorsys
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

In [2]:
# Global variables

# Path to directories
IMGS_DIR = Path("images")
OUTPUT_DIR = Path("output")

# Grid shape for coverage calculation
GRID_SHAPE = (3, 3)

In [3]:
# Read image paths
imgs_paths: list[Path] = []
for file in Path(IMGS_DIR).iterdir():
    if file.is_file() and file.suffix.lower() in {".jpg", ".png", ".jpeg"}:
        imgs_paths.append(file)

In [4]:
def get_distinct_colors(amount: int) -> list[tuple[int, int, int]]:
    """
    Generates a list of unique and distinct colors.
    Each color is represented as an RGB tuple.
    """
    colors = []
    for i in range(amount):
        # Evenly distribute hues in the HSV color space
        hue = i / amount
        saturation = 1.0  # Full saturation for vibrant colors
        value = 1.0  # Full brightness
        # Convert HSV to RGB and scale to 0-255
        rgb = colorsys.hsv_to_rgb(hue, saturation, value)
        rgb_scaled = tuple(int(c * 255) for c in rgb)
        colors.append(rgb_scaled)

    return colors

In [5]:
def display_image(image: np.ndarray) -> None:
    # Reset the figure
    plt.clf()

    plt.figure(figsize=(10, 10))
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.axis("off")
    plt.show()


def save_image(image: np.ndarray, path: Path) -> None:
    cv2.imwrite(str(path), image)
    print(f"Saved image to {path}")

In [6]:
def align_images_orb(reference: np.ndarray, target: np.ndarray) -> np.ndarray:
    """
    Aligns img_to_align to base_img using ORB feature matching.
    Returns the aligned image.
    """

    # Detect ORB features and compute descriptors.
    orb = cv2.ORB_create(5000)
    keypoints1, descriptors1 = orb.detectAndCompute(reference, None)
    keypoints2, descriptors2 = orb.detectAndCompute(target, None)

    # Check if descriptors were found
    if descriptors1 is None or descriptors2 is None:
        raise ValueError(
            f"No keypoints found in image #{1 if descriptors1 is None else 2}."
        )

    # Match features.
    matcher = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    matches = matcher.match(descriptors1, descriptors2)
    matches = sorted(matches, key=lambda x: x.distance)

    # Ensure there are enough matches
    if len(matches) < 4:
        raise ValueError("Not enough matches found to compute homography.")

    # Extract matched points.
    pts1 = np.float32([keypoints1[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
    pts2 = np.float32([keypoints2[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)

    # Compute homography.
    H, _ = cv2.findHomography(pts2, pts1, cv2.RANSAC)
    aligned_img = cv2.warpPerspective(
        target, H, (reference.shape[1], reference.shape[0])
    )
    return aligned_img


def align_images_ecc(reference: np.ndarray, target: np.ndarray) -> np.ndarray:
    """
    Aligns img_to_align to base_img using ECC (Enhanced Correlation Coefficient).
    Returns the aligned image.
    """
    # Convert images to grayscale
    gray_ref = cv2.cvtColor(reference, cv2.COLOR_BGR2GRAY)
    gray_target = cv2.cvtColor(target, cv2.COLOR_BGR2GRAY)

    # Define the number of iterations and termination criteria
    number_of_iterations = 5000
    termination_eps = 1e-10
    criteria = (
        cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT,
        number_of_iterations,
        termination_eps,
    )

    # Define the motion model (translation + rotation)
    warp_mode = cv2.MOTION_EUCLIDEAN

    # Initialize the warp matrix
    warp_matrix = np.eye(2, 3, dtype=np.float32)

    # Apply ECC algorithm to find the optimal warp matrix
    cc, warp_matrix = cv2.findTransformECC(
        gray_ref, gray_target, warp_matrix, warp_mode, criteria
    )

    # Apply the warp matrix to align the target image with the reference image
    aligned_img = cv2.warpAffine(
        target,
        warp_matrix,
        (reference.shape[1], reference.shape[0]),
        flags=cv2.INTER_LINEAR + cv2.WARP_INVERSE_MAP,
    )
    return aligned_img

In [ ]:
def normalize_images(img_paths: list[Path]) -> np.ndarray:
    """
    Takes a set of images and calculates a "normalized" image.
    """
    assert len(img_paths) > 1, "At least two images are required for normalization."
    images = []
    for img_path in img_paths:
        img = cv2.imread(str(img_path))
        if img is None:
            raise ValueError(f"Could not read image {img_path}.")
        # print(f"Read image {img_path} with shape {img.shape}.")
        images.append(img)

    assert all(img.shape == images[0].shape for img in images), (
        "All images must have the same resolution."
    )

    # Convert images to float32 for better precision in calculations.
    images_float = [img.astype(np.float32) for img in images]

    # Align all images to the first image
    aligned_images = [images_float[0]]
    for i, img in enumerate(images_float[1:]):
        try:
            print(f"Aligning image #{i + 2} to the first image...")
            aligned_img = align_images_ecc(images_float[0], img)
            aligned_images.append(aligned_img)
        except Exception as e:
            print(f"Error in alignment of image #{i + 2}: {e}")
            return None
    aligned_images = np.array(aligned_images)

    # Calculate the mean across all aligned images.
    mean_image = np.mean(aligned_images, axis=0).astype(np.uint8)
    display_image(mean_image)
    save_image(mean_image, OUTPUT_DIR / "normalized_image.png")

    return mean_image


image_paths = [
    IMGS_DIR / "Al_10D_0b.jpg",
    IMGS_DIR / "Al_10D_1.jpg",
    IMGS_DIR / "Al_10D_2.jpg",
    IMGS_DIR / "Al_10D_3.jpg",
    IMGS_DIR / "Al_10D_4.jpg",
]
_ = normalize_images(image_paths)

Aligning image #2 to the first image...


Timed out after >22m...